# Exercise - Add Memory and Self-reflection - STARTER

In this exercise, you’ll enhance your AI agent by adding self-reflection and memory. These features allow the agent to iteratively critique its responses and improve over time while maintaining a log of all interactions. 

This mimics how human learning and feedback loops work, pushing your agent towards more refined and accurate outputs.

**Challenge**

You are tasked with upgrading the existing agent. This version can learn from its previous answers, identify mistakes, and refine its responses automatically.

## 0. Import the necessary libs

In [1]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage

## 1. Recap: how to use OpenAI client with your API Key

To be able to connect with OpenAI, you need to instantiate an OpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
client = OpenAI(api_key="voc-")
```

In [ ]:
# TODO - Instantiate your client
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# OPENAI_API_KEY = "voc-*"
# client = OpenAI(
#     api_key = OPENAI_API_KEY
# )

In [3]:
response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer all user questions"},
            {"role": "user", "content":"What have I asked?"},
        ],
        temperature=0.0,
    )
response.choices[0].message.content

"I'm unable to recall previous interactions or questions you've asked. However, I'm here to help with any questions or topics you'd like to discuss now! What can I assist you with today?"

## 2. Recap: Adding Memory

In order to add reflection, you need to make sure your agent can keep  track of all interactions. Let's quickly recap how to do it with a simple list.

In [4]:
memory = [
    {"role": "system", "content": "Answer all user questions"},
    {"role": "user", "content": "What's an API"},
]

In [5]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nFor example, a weather application might use an API to retrieve weather data from a remote server. The API specifies how the application can request the data (e.g., through specific URLs and parameters) and what format the data will be returned in (e.g., JSON or XML). This allows developers to build applications that can leverage existing services and data, promoting interoperability and efficiency in software development.'}]

In [6]:
memory.append(
    {"role": "user", "content": "What have I asked?"}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nFor example, a weather application might use an API to retrieve weather data from a remote server. The API specifies how the application can request the data (e.g., through specific URLs and parameters) and what format the data will be returned in (e.g., JSON or XML). This allows developers to build applications that can leverage existing services and data, promoting interoperability and efficiency in software development.'},
 {'role

In [7]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.0,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

[{'role': 'system', 'content': 'Answer all user questions'},
 {'role': 'user', 'content': "What's an API"},
 {'role': 'assistant',
  'content': 'An API, or Application Programming Interface, is a set of rules and protocols that allows different software applications to communicate with each other. It defines the methods and data formats that applications can use to request and exchange information. APIs enable developers to access the functionality of other software services, libraries, or platforms without needing to understand their internal workings.\n\nFor example, a weather application might use an API to retrieve weather data from a remote server. The API specifies how the application can request the data (e.g., through specific URLs and parameters) and what format the data will be returned in (e.g., JSON or XML). This allows developers to build applications that can leverage existing services and data, promoting interoperability and efficiency in software development.'},
 {'role

## 3. Create a memory layer

Now that you remember how to use a list of messages, it's recommended to have a proper class to deal with more complicated cases.

Create the Memory class and add the following methods to it:
- add_message
- get_messages
- last_message

In [ ]:
# TODO - Create your memory layer
class Memory:
    def __init__(self):
        self.messages = []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get_messages(self):
        return self.messages

    def last_message(self):
        return self.messages[-1]

## 4. Update the Agent class

In this exercise, you will enhance the AI Agent with self-reflection capabilities, allowing it to critique its own responses and refine them iteratively. This feature enables the agent to evaluate its output and improve the response quality before delivering a final answer.

**Objective**

Your task is to modify the agent so that it can:

- Store conversation history – Implement a memory mechanism to track interactions.
- Generate an initial response – Process user input and return a response using the language model.
- Critique its own response when enabled – If self-reflection is activated, the agent should generate feedback on its own answer.
- Refine its response iteratively – Based on the self-critique, the agent should adjust its reply, improving clarity, accuracy, and relevance.

**Steps**

- Implement a memory layer to retain conversation history.
- Introduce a self-reflection mechanism that allows the agent to analyze its response and refine it.
- Limit the number of self-reflection iterations to prevent excessive loops (minimum 1, maximum 3).
- Ensure flexibility by allowing users to toggle self-reflection on or off.

**Considerations**

- The agent should always generate at least one response before self-reflection.
- If self-reflection is enabled, it should run at least once more to critique and improve its output.
- The number of iterations should be controlled and not exceed three refinements.
- Implement logging functionality (verbose mode) to track the refinement process.

**Invoke**

Refactor `invoke()` method. This method now should include:
- self_reflection paramenter (default: False);
- max_iter parameter (default: 1);

If self_reflection is set to True, it should use a loop to generate an initial response. Then critiquing and refining the response in subsequent iterations up to the number of iterations defined in max_iter.

Use the self.memory to store each step.

Rules for self-reflection:

- Don't allow values less than 1
- Don't allow values greater than 3
- Max iter is controlled by self_reflection flag. 
- If set to true, it needs to call the LLM at least once more for the criticism

Your self critique prompt should start with something like: `Reflect on your previous response`.
Extend it to make sure it identifies errors and provides a revised version.

In [ ]:
# TODO - Create your critique prompt
SELF_CRITIQUE_PROMPT = """Reflect on your previous response.
Analyze your previous response based on the previous input prompt.
Identify errors if there are any. If there are errors create a revised response.
Repeat this process at least once and until the analysis of the response does
not return any more errors, but maximally only as often as the provided
maximum number of iterations 'max_iter'.
Return the revised response in a JSON output structure:
{
"original_response:", "",
"revisions_needed:", "",
"updated_response:", ""
}
"""

In [ ]:
class Agent:
    """A self-reflection AI Agent"""

    def __init__(
        self,
        name:str = "Agent",
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        self_reflection:bool = False,
        max_iter: int = 1,
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")),
        # client = OpenAI(
        #     api_key = OPENAI_API_KEY
        # ),
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature

        # TODO - Instantiate your client properly
        self.client = client

        # TODO - Create your memory layer
        self.memory = Memory()
        self.memory.add_message("system", f"You are a {self.role}. {self.instructions}.")

        # TODO - Create your critique prompt
        self.critique_prompt = SELF_CRITIQUE_PROMPT

    def invoke(
        self,
        user_message: str,
        self_reflection: bool = False,
        max_iter: int = 1,
        verbose: bool = False
        ) -> str:

        # TODO - refactor the invoke method to add self-reflection
        # Rules
        # - Don't allow values less than 1
        # - Don't allow values greater than 3
        # - Max iter is controlled by self_reflection flag.
        # - If set to true, it needs to call the LLM at least once more for the criticism

        print(f"Self reflection was set to {self_reflection}.")

        self.memory.add_message("user", user_message)
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.memory.get_messages()
        )
        ai_message = response.choices[0].message.content


        self.memory.add_message("assistant", ai_message)

        if self_reflection:

            if max_iter < 1:
                print(f"The maximum number of iterations was originally selected as max_iter = {max_iter} < 1 and is reset to max_iter = 1.")
                max_iter = 1

            if max_iter > 3:
                print(f"The maximum number of iterations was originally selected as max_iter = {max_iter} > 3 and is reset to max_iter = 3.")
                max_iter = 3

            for i in range(max_iter):
                self.memory.add_message("user", self.critique_prompt)
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=self.memory.get_messages()
                )
                ai_message = response.choices[0].message.content
                self.memory.add_message("assistant", ai_message)

                if verbose:
                    print()
                    print("Compare with the last response:", self.memory.last_message())

        return ai_message


## 5. Build some agents and have fun

Create some specific agents and invoke them with self_reflection = True

In [ ]:
# TODO - create a default agent with role and instructions
# Then ask it a subjective question like:
# "Pick only one. Who is the best character in Game of Thrones?"

name = "Film Afficionado"
role = "Experienced Film Critique"
instructions = "Provide feedback to users on movie content, characters, background story, actors and other important movie information."

agent = Agent(name=name, role=role, instructions=instructions)

user_message = "Pick only one. Who is the best character in Game of Thrones?"

agent.invoke(
    user_message = user_message,
    self_reflection = True,
    max_iter = 1,
    verbose = True,
)

Self reflection was set to True.

Compare with the last response: {'role': 'assistant', 'content': '{\n  "original_response": "Choosing the best character in \'Game of Thrones\' is a subjective exercise, as the series is rich with complex characters, each with their own arcs and moral ambiguities. However, one standout character is Tyrion Lannister, portrayed by Peter Dinklage.\\n\\nCharacter Depth: Tyrion is a master of wit and intellect, often using his sharp tongue to navigate a world that looks down upon him due to his dwarfism. His intelligence creates a striking contrast with many of the often brutal characters surrounding him, and his journey from an underestimated member of House Lannister to a key political player resonates deeply with audiences.\\n\\nBackground Story: Raised in a family that between love and resentment, Tyrion\'s backstory adds layers to his character. He feels like an outsider, largely because of his father\'s intolerance and his siblings\' complicated loyal

'{\n  "original_response": "Choosing the best character in \'Game of Thrones\' is a subjective exercise, as the series is rich with complex characters, each with their own arcs and moral ambiguities. However, one standout character is Tyrion Lannister, portrayed by Peter Dinklage.\\n\\nCharacter Depth: Tyrion is a master of wit and intellect, often using his sharp tongue to navigate a world that looks down upon him due to his dwarfism. His intelligence creates a striking contrast with many of the often brutal characters surrounding him, and his journey from an underestimated member of House Lannister to a key political player resonates deeply with audiences.\\n\\nBackground Story: Raised in a family that between love and resentment, Tyrion\'s backstory adds layers to his character. He feels like an outsider, largely because of his father\'s intolerance and his siblings\' complicated loyalties. This background fuels not only his cunning but also his empathy; he seems more in touch with 

In [21]:
agent.memory.get_messages()

[{'role': 'system',
  'content': 'You are a Experienced Film Critique. Provide feedback to users on movie content, characters, background story, actors and other important movie information..'},
 {'role': 'user',
  'content': 'Pick only one. Who is the best character in Game of Thrones?'},
 {'role': 'assistant',
  'content': 'Choosing the best character in "Game of Thrones" is a subjective exercise, as the series is rich with complex characters, each with their own arcs and moral ambiguities. However, one standout character is Tyrion Lannister, portrayed by Peter Dinklage.\n\n**Character Depth**: Tyrion is a master of wit and intellect, often using his sharp tongue to navigate a world that looks down upon him due to his dwarfism. His intelligence creates a striking contrast with many of the often brutal characters surrounding him, and his journey from an underestimated member of House Lannister to a key political player resonates deeply with audiences.\n\n**Background Story**: Raised i

In [22]:
agent.memory.last_message()

{'role': 'assistant',
 'content': '{\n  "original_response": "Choosing the best character in \'Game of Thrones\' is a subjective exercise, as the series is rich with complex characters, each with their own arcs and moral ambiguities. However, one standout character is Tyrion Lannister, portrayed by Peter Dinklage.\\n\\nCharacter Depth: Tyrion is a master of wit and intellect, often using his sharp tongue to navigate a world that looks down upon him due to his dwarfism. His intelligence creates a striking contrast with many of the often brutal characters surrounding him, and his journey from an underestimated member of House Lannister to a key political player resonates deeply with audiences.\\n\\nBackground Story: Raised in a family that between love and resentment, Tyrion\'s backstory adds layers to his character. He feels like an outsider, largely because of his father\'s intolerance and his siblings\' complicated loyalties. This background fuels not only his cunning but also his emp

In [23]:
agent.memory.last_message()["content"]

'{\n  "original_response": "Choosing the best character in \'Game of Thrones\' is a subjective exercise, as the series is rich with complex characters, each with their own arcs and moral ambiguities. However, one standout character is Tyrion Lannister, portrayed by Peter Dinklage.\\n\\nCharacter Depth: Tyrion is a master of wit and intellect, often using his sharp tongue to navigate a world that looks down upon him due to his dwarfism. His intelligence creates a striking contrast with many of the often brutal characters surrounding him, and his journey from an underestimated member of House Lannister to a key political player resonates deeply with audiences.\\n\\nBackground Story: Raised in a family that between love and resentment, Tyrion\'s backstory adds layers to his character. He feels like an outsider, largely because of his father\'s intolerance and his siblings\' complicated loyalties. This background fuels not only his cunning but also his empathy; he seems more in touch with 

In [24]:
p = json.loads(agent.memory.last_message()["content"])
print(p)

{'original_response': "Choosing the best character in 'Game of Thrones' is a subjective exercise, as the series is rich with complex characters, each with their own arcs and moral ambiguities. However, one standout character is Tyrion Lannister, portrayed by Peter Dinklage.\n\nCharacter Depth: Tyrion is a master of wit and intellect, often using his sharp tongue to navigate a world that looks down upon him due to his dwarfism. His intelligence creates a striking contrast with many of the often brutal characters surrounding him, and his journey from an underestimated member of House Lannister to a key political player resonates deeply with audiences.\n\nBackground Story: Raised in a family that between love and resentment, Tyrion's backstory adds layers to his character. He feels like an outsider, largely because of his father's intolerance and his siblings' complicated loyalties. This background fuels not only his cunning but also his empathy; he seems more in touch with the struggles 

In [25]:
json.loads(agent.memory.last_message()["content"])["updated_response"]

"Selecting the best character in 'Game of Thrones' is subjective, reflecting personal preferences shaped by individual experiences and values. However, a compelling choice is Tyrion Lannister, portrayed by Peter Dinklage.\n\n**Character Depth**: Tyrion is a master of wit and intellect, using his sharp tongue to navigate a world that often looks down on him due to his dwarfism. His intelligence contrasts sharply with many brutal characters, and his journey from an underestimated Lannister to a key political player resonates deeply with audiences.\n\n**Background Story**: Raised in a family filled with both love and resentment, Tyrion's background adds layers to his character. Feeling like an outsider because of his father's intolerance and his siblings' complexities fuels not just his cunning but also a sense of empathy; he relates more to common people's struggles than many nobles do.\n\n**Actor's Performance**: Peter Dinklage’s performance brings a blend of vulnerability, humor, and g

## 6. Experiment

Now that you understood how it works, experiment with new things.

- Experiment new critique prompts
- What happens when you increase the number of iterations?
- Try adding an argument to invoke() method to inspect it (verbose=True)
- What else can you try?